In [ ]:
import os
import numpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
from itertools import cycle
import seaborn as sns
import statsmodels.api as sm 
from scipy.interpolate import interp1d
from prophet import Prophet

pd.set_option('display.max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
%%time
oj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
train = pd.read_csv(oj(INPUT_DIR, 'trainset.csv'))
price = pd.read_csv(oj(INPUT_DIR, 'sell_prices.csv'))
calendar = pd.read_csv(oj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(oj(INPUT_DIR, 'submit.csv'))

In [ ]:
print(calendar['event_name_1'].unique())

In [ ]:
print(calendar['event_type_1'].unique())

In [ ]:
print(calendar['event_name_2'].unique())

In [ ]:
print(calendar['event_type_2'].unique())

In [ ]:
print(price.head())
print(price.shape)

In [ ]:
print(train.shape)

In [ ]:
holidays = calendar[['date', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']]
holidays = holidays.rename(columns={'date': 'ds'})
holidays['lower_window'] = 0
holidays['upper_window'] = 1
holidays1 = holidays.dropna(subset = ['event_name_1'])
holidays2 = holidays.dropna(subset = ['event_name_2'])
holidays1 = holidays1[['ds', 'lower_window', 'upper_window', 'event_name_1']]
holidays1 = holidays1.rename(columns={'event_name_1': 'holiday'})
holidays2 = holidays2[['ds', 'lower_window', 'upper_window', 'event_name_2']]
holidays2 = holidays2.rename(columns={'event_name_2': 'holiday'})
holidays = pd.concat((holidays1, holidays2))
holidays = holidays.reset_index(drop=True)
print(holidays.head())



In [ ]:
for i in range(train.shape[0]):
    df = pd.DataFrame(calendar[['date', 'd', 'wm_yr_wk', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']])
    df_train =  df.loc[0:1926]
    df_test = df.loc[1927:]
    #print(df_train)
    #print(df_test)
    df_train['y'] = train.loc[i,'d_1':].values
    df_train = pd.merge(df_train, price[(price['store_id'] == train.loc[i,'store_id']) & (price['item_id'] == train.loc[0,'item_id'])][['wm_yr_wk', 'sell_price']], how='left', on=['wm_yr_wk'])
    df_train = df_train.rename(columns={'date': 'ds'})
    #df_train = df_train.drop(["price"], axis=1)
    print(df_train)
    print("-------------------")
    
    model = Prophet()
    model.fit(df_train)
    future = model.make_future_dataframe(periods=14)
    print(future)
    forecast = model.predict(future)
    print(forecast.tail(14))
    model.plot(forecast.tail(14))
    plt.show()
    #df_train.to_csv('df_train' + str(i) + '.csv', index=False)